In [62]:
!kaggle datasets download -d sherrytp/airline-delay-analysis -f "airline delay analysis/2018.csv"
!unzip 2018.csv.zip
!rm 2018.csv.zip

 99%|███████████████████████████████████████▋| 236M/238M [00:05<00:00, 44.7MB/s]
100%|████████████████████████████████████████| 238M/238M [00:05<00:00, 42.7MB/s]
Archive:  2018.csv.zip
replace 2018.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [63]:
# initlize pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("airline-delay-regression").getOrCreate()

ConnectionRefusedError: [Errno 111] Connection refused

In [4]:
# Needed to make Jupyter work with Gitpod
import plotly.io as pio
pio.renderers.default = 'iframe_connected'

In [60]:
# Read the data into a dataframe and print the schema
df = spark.read.csv("2018.csv", header=True, inferSchema=True)
df.printSchema()

ConnectionRefusedError: [Errno 111] Connection refused

In [56]:
# import matplotlib.pyplot as plt

# df.plot.scatter(x='FL_DATE', y='ARR_DELAY')

ValueError: Unable to parse datatype from schema. [Errno 111] Connection refused

In [58]:
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import udf

df = df.drop("Unnamed: 27", "LATE_AIRCRAFT_DELAY", "SECURITY_DELAY", "NAS_DELAY", "WEATHER_DELAY", "CARRIER_DELAY", "AIR_TIME", "ACTUAL_ELAPSED_TIME", "DIVERTED", "CANCELLATION_CODE", "CANCELLED", "ARR_TIME", "TAXI_IN", "WHEELS_ON", "WHEELS_OFF", "TAXI_OUT", "DEP_TIME", "OP_CARRIER_FL_NUM", "ARR_DELAY")

df.printSchema()

@udf(returnType=IntegerType())
def get_month(date):
    return int(date.split("-")[1])

@udf(returnType=IntegerType())
def get_day(date):
    return int(date.split("-")[2])

@udf(returnType=IntegerType())
def get_year(date):
    return int(date.split("-")[0])

# Adds month and a day column to the dataframe
df = df.withColumn("YEAR", get_year(df["FL_DATE"]).cast(IntegerType()))
df = df.withColumn("MONTH", get_month(df["FL_DATE"]).cast(IntegerType()))
df = df.withColumn("DAY", get_day(df["FL_DATE"]).cast(IntegerType()))

df = df.drop("FL_DATE")

df = df.dropna()

df.printSchema()

# Print the first 5 rows of the dataframe
df.show(5)
print(df.count())

ConnectionRefusedError: [Errno 111] Connection refused

In [47]:
# Preparing the data
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import OneHotEncoder

train, test = df.randomSplit([0.8, 0.2])

# Encode the categorical features using StringIndexer
indexer = StringIndexer(inputCols=["OP_CARRIER", "ORIGIN", "DEST", "DAY", "MONTH"], outputCols=["OP_CARRIER_INDEX", "ORIGIN_INDEX", "DEST_INDEX", "DAY_INDEX", "MONTH_INDEX"])

# Use one hot encoding to encode the categorical features
encoder = OneHotEncoder(inputCols=["OP_CARRIER_INDEX", "ORIGIN_INDEX", "DEST_INDEX", "DAY_INDEX", "MONTH_INDEX"], outputCols=["OP_CARRIER_VEC", "ORIGIN_VEC", "DEST_VEC", "DAY_VEC", "MONTH_VEC"])

# Create the assembler
assembler = VectorAssembler(inputCols=["OP_CARRIER_VEC", "ORIGIN_VEC", "DEST_VEC", "CRS_DEP_TIME", "CRS_ARR_TIME", "DAY_VEC", "MONTH_VEC", "YEAR"], outputCol="features")

# First Classifier: Linear Regression

In [48]:
# Linear Regression
from pyspark.ml.regression import LinearRegression
from pyspark.ml.pipeline import Pipeline
from pyspark.ml.evaluation import RegressionEvaluator


lr = LinearRegression(featuresCol="features", labelCol="DEP_DELAY", regParam=0.3)

# Create the pipeline
pipeline = Pipeline(stages=[indexer, encoder, assembler, lr])

# Train the model
model = pipeline.fit(train)

# Evaluate the model
predictions = model.transform(test)

# Print the accuracy, RMSE, and MAE
evaluator = RegressionEvaluator(predictionCol="prediction", labelCol="DEP_DELAY", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("RMSE:", rmse)

RMSE: 44.274917223028034


In [44]:
print("Mean:", test.select("DEP_DELAY").agg({"DEP_DELAY": "mean"}).collect()[0][0])
print("Std:", test.select("DEP_DELAY").agg({"DEP_DELAY": "stddev"}).collect()[0][0])




Mean: 9.93221958518358


Std: 44.739548581249586
